# Homework 4

## First Part: Does basic house information reflect house's description?

The goal of this part of the homework, consists in implement two clustering, based on two differents criteria, but using the same data, and compare the results.

### Scraping

For getting out data, we scrap the information from the site www.inmobiliare.it. For each appartament we took information of: title, link, price, locali, superficie, bagni, piano and description and return a csv called 'immobiliare2.csv'. 

For more details about the process of the scrapping, consult the 'scraper.py' file, available in the respository.

In [6]:
import pandas as pd

In [10]:
# Raw data after the srapping process
df = pd.read_csv("C:\Users\falli\Downloads\ADM_HW4-masterimmobiliare2.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-10-2dfe593b0063>, line 2)

### Preprocessing

In order to have usefull information for the following analisys, we clean the data:

On the text varibles:
- drop NA
- remove punctuation
- remove stopwords
- steam 

On the numeric variables:

- drop NA
- drop characters no numeric as '€' among others
- replace characters as "T" in the column "piano", for 0

In [12]:
# import preprocessing

In [ ]:
p = Preprocesser(df)
p.preprocess()
df.to_csv("immobiliare2preprocessed.csv")

### Datasets

Now, with the dataframe, preproceced, we proceed to create the two databases needed, tha information matrix and the description matrix.

In [ ]:
# import matrix builder

### Clustering

#### Similar clusters

#### Word cloud of house descriptions